# Multilingual Twitter sentiment analysis with BERT

This notebook assumes that the clean dataset is stored on google drive in "clean" folder.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15.0
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 412.3MB 37kB/s 
     |████████████████████████████████| 512kB 47.1MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 39.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=e321e32d1baeddbc269db36b99a3544caa44ef216dd505dc56ebe6d17f7d6d9f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import io

from datetime import datetime

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
# Set the output directory for saving model file

OUTPUT_DIR = 'output'#@param {type:"string"}
tf.gfile.MakeDirs(OUTPUT_DIR)
print('Model output directory: {}'.format(OUTPUT_DIR))


Model output directory: output


#Data

In [5]:
DATA_COLUMN = 'Text'
LABEL_COLUMN = 'HandLabels'
label_list = ['Positive', 'Negative', 'Neutral']

path = '/content/drive/My Drive/clean/'
languages = ['Hungarian', 'Portuguese', 'Bosnian',
             'Croatian', 'Polish', 'Russian',
             'Serbian', 'Slovak', 'Slovenian',
             'English', 'German', 'Swedish']

In [6]:
def load_dataset(train_langs, test_langs, train_on_test_lang=True):
    
    # Load train languages
    df_train = pd.DataFrame()
    for lang in train_langs:
        df = pd.read_csv(path + lang + '.csv')
        df.drop('Unnamed: 0', axis=1, inplace=True)
        df.dropna(axis=0, inplace=True)
        df_train = pd.concat([df_train, df], ignore_index=True)
    
    df_train.reset_index(drop=True, inplace=True)
    
    # Load test language
    df_test = pd.read_csv(path + test_lang + '.csv')
    df_test.drop('Unnamed: 0', axis=1, inplace=True)
    df_test.dropna(axis=0, inplace=True)
    df_test.reset_index(drop=True, inplace=True)
    
    if train_on_test_lang:
        train_size = 0.3
        num_train = int(df_test.shape[0] * train_size)
        df_train = pd.concat([df_train, df_test.head(num_train)], ignore_index=True)
        df_test = df_test.tail(df_test.shape[0]-num_train)
    
    return df_train, df_test

In [7]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"], tokenization_info["do_lower_case"]])
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

#Creating a model
1. Load BERT
2. Create a single new layer for finetunning


In [8]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
    
    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [9]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                #avg_f1 = tf.contrib.metrics.f1_score(label_ids, predicted_labels)

                return {
                    "eval_accuracy": accuracy,
                    #"eva_avg_f1": avg_f1,
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


In [10]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 10000
SAVE_SUMMARY_STEPS = 10000

In [11]:
def fit_and_evaluate(train, test):
    # Use the InputExample class from BERT's run_classifier code to create examples from the data
    train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                       text_a = x[DATA_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)

    test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                       text_a = x[DATA_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)
    
    # We'll set sequences to be at most 128 tokens long.
    MAX_SEQ_LENGTH = 128
    # Convert our train and test features to InputFeatures that BERT understands.
    train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
    test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
    
    
    # Compute # train and warmup steps from batch size
    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
    
    # Specify output directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)
    
    
    model_fn = model_fn_builder(
        num_labels=len(label_list),
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params={"batch_size": BATCH_SIZE})

    # Create an input function for training.
    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)
    
    
    print(f'Beginning Training!')
    current_time = datetime.now()
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Training took time ", datetime.now() - current_time)
    
    
    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)
    
    results = estimator.evaluate(input_fn=test_input_fn, steps=None)
    print(results)
    with open('/content/drive/My Drive/results/results.txt', 'a+') as f:
        f.write("{}\n".format(results))

In [12]:
experiments_same_fam = [
    (['English'], 'German'),
    (['Serbian'], 'Slovenian'),
    (['Serbian'], 'Croatian'),
    (['Serbian'], 'Bosnian'),
    (['Polish'], 'Slovenian'),
    (['Slovak'], 'Slovenian'),
    (['Croatian'], 'Slovenian'),
    (['Croatian'], 'Serbian'),
    (['Croatian'], 'Bosnian'),
    (['Slovenian'], 'Croatian'),
    (['Slovenian'], 'Serbian'),
    (['Slovenian'], 'Bosnian'),
]

In [13]:
experiments_diff_lang_fam = [
    (['German'], 'Slovenian'),
    (['English'], 'Slovenian'),
    (['Swedish'], 'Slovenian'),
    (['Hungarian'], 'Slovenian'),
    (['Portuguese'], 'Slovenian'),
]

In [14]:
experiments_large_train_dataset = [
    (['Croatian', 'Serbian', 'Bosnian'], 'Slovenian'),
    (['English', 'Swedish'], 'German'),
]

In [ ]:
for (train_langs, test_lang) in experiments_same_fam:
    with open('/content/drive/My Drive/results/results_mbert.txt', 'a+') as f:
        f.write("{} {} \n".format(train_langs, test_lang))
    print(train_langs, test_lang)
    df_train, df_test = load_dataset(train_langs, test_lang)
    fit_and_evaluate(df_train, df_test)

In [ ]:
for (train_langs, test_lang) in experiments_diff_lang_fam:
    with open('/content/drive/My Drive/results/results_mbert.txt', 'a+') as f:
        f.write("{} {} \n".format(train_langs, test_lang))
    print(train_langs, test_lang)
    df_train, df_test = load_dataset(train_langs, test_lang)
    fit_and_evaluate(df_train, df_test)

In [ ]:
for (train_langs, test_lang) in experiments_large_train_dataset:
    with open('/content/drive/My Drive/results/results_mbert.txt', 'a+') as f:
        f.write("{} {} \n".format(train_langs, test_lang))
    print(train_langs, test_lang)
    df_train, df_test = load_dataset(train_langs, test_lang)
    fit_and_evaluate(df_train, df_test)